Let's do a quick example comparing `jitr` to the standard Runge-Kutta ODE solver implemented in `scipy.integrate.solve_ivp`:

In [1]:
import numpy as np
from scipy.integrate import solve_ivp
from numba import njit
from jitr import reactions, rmatrix, utils
from jitr.reactions.potentials import woods_saxon_potential

First let's check our library versions and configs, as this will affect performance:

In [2]:
import scipy as sc

sc.__version__

'1.13.0'

In [3]:
import numba

numba.__version__

'0.60.0'

In [4]:
np.__version__

'1.26.4'

In [5]:
# np.show_config()

Great, now let's set up our system and solver with `jitr`:

In [6]:
sys = reactions.ProjectileTargetSystem(
    channel_radius=10 * np.pi,
    lmax=10,
    mass_target=44657,
    mass_projectile=938.3,
    Ztarget=40,
    Zproj=0,
)

# COM frame energy
Elab = 14.1

# Lagrange-Mesh R-matrix solver
solver = rmatrix.Solver(40)

# channels holds info for the elastic scattering channel
Elab = 42.1
mu, Ecm, k, eta = utils.kinematics.classical_kinematics(
    sys.mass_target, sys.mass_projectile, Elab, sys.Zproj * sys.Ztarget
)
channels, asymptotics = sys.get_partial_wave_channels(Ecm, mu, k, eta)

In [7]:
# Woods-Saxon potential parameters
V0 = -60  # real potential strength
W0 = -20  # imag potential strength
R0 = 4  # Woods-Saxon potential radius
a0 = 0.5  # Woods-Saxon potential diffuseness
params = (V0, W0, R0, a0)

In [8]:
# run solver for S-wave
l = 0
R, S, uext_prime_boundary = solver.solve(
    channels[l], asymptotics[l], woods_saxon_potential, params
)

In [9]:
print(S[0, 0])

(0.16908560771865788+0.05598287996028857j)


Great, now let's use `scipy` and see if we get the same $\mathcal{S}$-matrix:

In [10]:
# Runge-Kutta
from jitr.utils import schrodinger_eqn_ivp_order1

channel_data_rk = reactions.make_channel_data(channels[l])
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol

In [11]:
a = channel_data_rk[0].domain[1]
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)
print(S_rk)

(0.1694677812223941+0.055846727056314265j)


In [12]:
100 * (S[0, 0] - S_rk) / S_rk  # percent difference in real and imag parts of S

(-0.17954038594229624+0.13950754034898027j)

Great, our solvers agree up to high precision. Now let's compare the runtime of the two solver options:

In [13]:
%%timeit
l = 0
R, S, uext_prime_boundary = solver.solve(
    channels[l], asymptotics[l], woods_saxon_potential, params
)

320 µs ± 16.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
domain, init_con = channel_data_rk[0].initial_conditions()
sol_rk = solve_ivp(
    lambda s, y,: schrodinger_eqn_ivp_order1(
        s, y, channel_data_rk[0], woods_saxon_potential, params
    ),
    domain,
    init_con,
    dense_output=True,
    atol=1.0e-7,
    rtol=1.0e-7,
).sol
R_rk = sol_rk(a)[0] / (a * sol_rk(a)[1])
S_rk = utils.smatrix(R_rk, a, channel_data_rk[0].l, channel_data_rk[0].eta)

76.7 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


On my machine `jitr` is faster by about 250 times!

(This does, of course, depend on the solver paramaters; `atol` and `rtol` for `solve_ivp`, and `nbasis` for `LagrangeRMatrixSolver` ).